In [1]:
import os
os.chdir('..')

In [2]:
import torch
from layers.Embed import DataEmbedding

In [6]:
b, l, c = 32, 96, 7

input = torch.rand([b, l, c])
input_mark = torch.rand([b, l,4])

In [ ]:
embed = DataEmbedding(c, 256, embed_type='timeF', freq='h')

In [9]:
out = embed(input, input_mark)

In [10]:
out.shape

torch.Size([32, 96, 256])

In [5]:
def test_data_embedding():
    """
    DataEmbedding 模块的单元测试
    涵盖: 维度检查, timeF/fixed 两种模式, GPU/CPU 兼容性
    """
    print("=" * 20 + " 开始测试 DataEmbedding " + "=" * 20)
    
    # 1. 基础参数设置
    batch_size = 32
    seq_len = 96      # 常见的时间序列预测长度
    c_in = 7          # 输入变量个数 (例如: 7个气象指标)
    d_model = 512     # 隐层维度
    dropout = 0.1
    
    # 自动检测设备
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"🧩 测试设备: {device}")

    # ==========================================
    # 测试案例 A: embed_type='timeF' (数值型时间特征 - 推荐)
    # ==========================================
    print("\n[Case A] 测试 embed_type='timeF' (线性投影)...")
    
    # 对于 freq='h' (小时), TimeFeatureEmbedding 通常期待 4 个特征 (Month, Day, Weekday, Hour)
    # 输入必须是 float 类型
    x = torch.randn(batch_size, seq_len, c_in).to(device)
    x_mark_timef = torch.randn(batch_size, seq_len, 4).to(device) 
    
    try:
        model_timef = DataEmbedding(
            c_in=c_in, 
            d_model=d_model, 
            embed_type='timeF', 
            freq='h', 
            dropout=dropout
        ).to(device)
        
        output = model_timef(x, x_mark_timef)
        
        # 验证输出
        expected_shape = (batch_size, seq_len, d_model)
        assert output.shape == expected_shape, \
            f"Shape Mismatch! Expected {expected_shape}, got {output.shape}"
        assert output.requires_grad == True, "Output should verify gradients!"
        
        print(f"✅ Pass. Output Shape: {output.shape}")
        
    except Exception as e:
        print(f"❌ Fail. Error: {e}")
        # 如果是维度不匹配，通常是因为 TimeFeatureEmbedding 的 input_size 硬编码与输入不符
        print("💡 提示: 检查 TimeFeatureEmbedding 中的 freq_map 字典是否匹配输入 x_mark 的列数")

    # ==========================================
    # 测试案例 B: embed_type='fixed' (类别型时间嵌入 - 传统)
    # ==========================================
    print("\n[Case B] 测试 embed_type='fixed' (Embedding 查表)...")
    
    # TemporalEmbedding 在源码中通常硬编码切片 x[:,:,0] 到 x[:,:,4]
    # 所以即使 freq='h'，为了安全通常需要至少 5 列输入，或者代码内部需要做保护
    # 这里我们构造 5 列数据 (Month, Day, Weekday, Hour, Minute) 且必须是 Long/Int 类型
    x_mark_fixed = torch.randint(0, 12, (batch_size, seq_len, 5)).long().to(device)
    
    try:
        model_fixed = DataEmbedding(
            c_in=c_in, 
            d_model=d_model, 
            embed_type='fixed', 
            freq='h', 
            dropout=dropout
        ).to(device)
        
        output = model_fixed(x, x_mark_fixed)
        
        assert output.shape == (batch_size, seq_len, d_model)
        print(f"✅ Pass. Output Shape: {output.shape}")
        
    except IndexError:
        print("❌ Fail (IndexError).")
        print("💡 原因分析: TemporalEmbedding 代码中使用了 x[:,:,4]，要求 x_mark 至少有 5 列。")
    except Exception as e:
        print(f"❌ Fail. Error: {e}")

    # ==========================================
    # 测试案例 C: 无时间戳输入 (仅 Value + Positional)
    # ==========================================
    print("\n[Case C] 测试 x_mark=None...")
    
    try:
        output = model_timef(x, None) # 复用上面的模型
        assert output.shape == (batch_size, seq_len, d_model)
        print(f"✅ Pass. Output Shape: {output.shape}")
    except Exception as e:
        print(f"❌ Fail. Error: {e}")

    print("\n" + "=" * 20 + " 测试结束 " + "=" * 20)

test_data_embedding()

==================== 开始测试 DataEmbedding ====================
🧩 测试设备: cuda

[Case A] 测试 embed_type='timeF' (线性投影)...
✅ Pass. Output Shape: torch.Size([32, 96, 512])

[Case B] 测试 embed_type='fixed' (Embedding 查表)...
✅ Pass. Output Shape: torch.Size([32, 96, 512])

[Case C] 测试 x_mark=None...
✅ Pass. Output Shape: torch.Size([32, 96, 512])

==================== 测试结束 ====================
